# Stack Overflow Tag Imputation using Multi-label Classification

## Imports

In [6]:
# data structures
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_pandas
from collections import Counter
import string

# iterators
import itertools

# text processing
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

# data preprocessing
from sklearn.model_selection import train_test_split

# classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

# statistics
from random import randint

## Settings

In [3]:
# download
nltk.download('stopwords')
nltk.download('punkt')

# create and register a new `tqdm` instance with `pandas`
# tqdm_pandas(tqdm())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maksi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maksi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0it [00:00, ?it/s]


## Data Preprocessing

In [4]:
# to build and traing our model, we are going to have to work with posts have labeled tags
# posts = pd.read_csv("../../../../160-Stackoverflow-Data/300000_rows/Posts.csv", 
#                     dtype={'LastEditorDisplayName': str})
posts = pd.read_csv("../../../../160-Stackoverflow-Data/100000_rows/Posts.csv", 
                    dtype={'LastEditorDisplayName': str})
sub_posts = posts[['Id', 'Body', 'Title', 'Tags']]
sub_posts = sub_posts[~sub_posts['Tags'].isnull()]

In [5]:
print(sub_posts.shape)
sub_posts.head()

(50053, 4)


,Id,Body,Title,Tags
0,49104125,<p>Am new to jquery. \r\n I try to create m...,down caret rotating not working,<jquery><css><asp.net><html5>
1,49104126,<p>We are working on rewrite of an existing ap...,Parallel job execution with split-and-aggregat...,<java><multithreading><jobs>
2,49104127,"<p>In Graph Explorer, when logged in as a user...","Microsoft Graph: ""Unknown Error"" when calling ...",<azure><active-directory><microsoft-graph><msal>
6,49104131,<p>I'm currently using Google Drive SDK in my ...,Class implemented in both Derived Data and App...,<ios><swift><xcode><google-drive-sdk>
7,49104132,"<p>I have 3 Spinners citySpinner, regionSpinne...",How to filter spinner based on other spinner s...,<android><json>


## Data Cleaning

The body of our dataframe is encoded in html. Additionally, we will have to remove stop words.

In [11]:
class CleanText:
    STOPWORDS = set(stopwords.words('english'))
    URI_RE = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'
    lemmatizer = WordNetLemmatizer()

    @staticmethod
    def lemmatize(text):
        return ' '.join([CleanText.lemmatizer.lemmatize(w) for w in word_tokenize(text)])

    @staticmethod
    def strip_html_tags_and_uris(text):
        if text:
            soup = BeautifulSoup(text, "html.parser")
            # stripping all <code> tags with their content if any
            if soup.code:
                soup.code.decompose()
            # get all the text out of the html
            text = soup.get_text()
            # returning text stripping out all uris
            return re.sub(CleanText.URI_RE, "", text)
        else:
            return ""

    @staticmethod
    def lower_and_remove_punctuation(text):
        x = re.sub(r'[^\x00-\x7f]', r' ', text.lower())
        # replace punctuation with space
        return re.sub("[" + string.punctuation + "]", " ", x)

    @staticmethod
    def remove_stopwords(text):
        return ' '.join([w for w in word_tokenize(text) if w not in CleanText.STOPWORDS])


# sample function test
o = CleanText()
examples = ["SQL Server -{any equivalent of Excel's} CHOOSE function?",
            "How to free c++ memory vector<int> * (arr)?"]
for e in examples:
    print(o.lemmatize(e))
    print(o.strip_html_tags_and_uris(e))
    print(o.lower_and_remove_punctuation(e))
    print(o.remove_stopwords(e) + '\n')

SQL Server - { any equivalent of Excel 's } CHOOSE function ?
SQL Server -{any equivalent of Excel's} CHOOSE function?
sql server   any equivalent of excel s  choose function 
SQL Server - { equivalent Excel 's } CHOOSE function ?

How to free c++ memory vector < int > * ( arr ) ?
How to free c++ memory vector * (arr)?
how to free c   memory vector int     arr  
How free c++ memory vector < int > * ( arr ) ?



In [ ]:
# filter tags carrots
sub_posts['Tags'] = sub_posts['Tags'].apply(lambda t: ' '.join(re.findall(r"\<(\w+)\>", t)))

# filter title, and body of the question, as to be used in the context of the prediction
# self note: use progress_apply is an alternative to apply offered by tqdm
sub_posts['Title'] = sub_posts['Title'].apply(lambda t: CleanText.lower_and_remove_punctuation(t))
sub_posts['Title'] = sub_posts['Title'].apply(lambda t: CleanText.lemmatize(t))

sub_posts['Body'] = sub_posts['Body'].apply(lambda t: CleanText.strip_html_tags_and_uris(t))
sub_posts['Body'] = sub_posts['Body'].apply(lambda t: CleanText.lower_and_remove_punctuation(t))
sub_posts['Body'] = sub_posts['Body'].apply(lambda t: CleanText.lemmatize(t))
sub_posts['Body'] = sub_posts['Body'].apply(lambda t: CleanText.remove_stopwords(t))

In [ ]:
# sub_posts.to_csv('PostsClean.csv')
sub_posts.head()

# Building a Bag of Words Model

We will build a set of vectors $[i_0, i_1, i_2, \dots, i_k]$ where $k$ is an integer that maps to the word corpus and $i_k$ represents the frequency of the mapped word for each post. 

In [104]:
# first count all the words globally
word_count = Counter()
for body in sub_posts['Body']:
    word_count.update(Counter(body.split()))

# lets give titles twice the weight because they are more important
for title in sub_posts['Title']:
    word_count.update(Counter(title.split() + title.split()))

In [81]:
# how many unique tags are there?
tags_joined = list(itertools.chain.from_iterable([tag.split() for tag in sub_posts['Tags']]))
unique_tags = set(tags_joined)
print('Total number of unique tags', len(n_unique_tags))
print('Ration: unique tag to total unique words', len(unique_tags)/len(tags_joined))

Total number of unique tags 7805
Ration: unique tag to total unique words 0.07616491827274945


In [60]:
# sort the words by frequency and limit to the number of tags we have in total
most_common_words = sorted(word_count.items(), key=lambda x: -x[1])[:len(unique_tags)]

# print the first few as an example
count = 1
for key, value in most_common_words:
    if count < 5:
        print(key, ':', value)
        count += 1
    else:
        break

> : 951536
< : 948019
'' : 536876
& : 476602


## Extending with TF-IDF

We can extend the bag-of-words framework by taking into account frequencies of words in the corpora. This penalizes too frequent words and provides a better features space. 

In [61]:
# settings: 
# - filter rare words (5 or fewer occurances) 
# - filter too frequent words (occur more than in 90% of the time)
# - use bigrams along with unigrams in your vocabulary
# - (\S+) is a regex pattern that ignore all white spaces as delimitors
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern= '(\S+)')
X = tfidf_vectorizer.fit_transform(sub_posts['Title'].values)

In [62]:
X

<50053x10640 sparse matrix of type '<class 'numpy.float64'>'
	with 345904 stored elements in Compressed Sparse Row format>

In [91]:
# we need to make sure that our vocabulary contain some important key word
# we should be able to scout out like C++ and C#

# print to file, since too large to view
# with open("temp.txt", "w") as text_file:
#     print(tfidf_vectorizer.vocabulary_, file=text_file)

# build tag distribution
# tag_dist = pd.DataFrame.from_dict(Counter(tags_joined), orient='index').reset_index()
# tag_dist.columns = ['tag', 'frequency']
# tag_dist.to_csv('tag_distribution.csv')

## Setting Up a Multi-label Classifier

Our respond variable can have multiple tags. To deal with this kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s.

In [64]:
# transform the respond (tags) as a binary multilabel
mlb = MultiLabelBinarizer(classes=sorted(list(unique_tags)))
y_labels = [tag.split().values for tag in sub_posts['Tags'].values]
y = mlb.fit_transform(y_labels)

In [65]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Training the Classifier

We will use the One-vs-Rest approach, which compare a single class against everything else as the other class. As our classifier, we will use logistic regression.


In [66]:
# split into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# for viewing purposes
X_train_view, X_test_view, \
    y_train_view, y_test_view = train_test_split(sub_posts['Title'].values, y_labels, 
                                                 test_size=0.33, random_state=42)

# train the classifer
model_logistic = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 19 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 26 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 31 i

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 323 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 326 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 333 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 339 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 340 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 348 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 596 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 597 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 598 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 607 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 616 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 619 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 888 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 905 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 909 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 910 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 911 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 928 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1177 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1189 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1192 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1204 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1205 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1239 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

...

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6328 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6332 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6340 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6343 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6352 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6357 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6655 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6658 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6665 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6666 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6670 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 6689 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7005 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7010 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7011 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7012 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7017 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7021 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7322 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7327 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7334 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7339 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7359 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7363 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7681 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7686 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7698 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7705 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7707 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 7712 is present in all training examples.
  str(classes[c]))
C:\Users\maksi\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

## Evaluating Performance

In [67]:
# predict using the binary multilabels
predicted_labels_tfidf = model_logistic.predict(X_test)
predicted_scores_tfidf = model_logistic.decision_function(X_test)

# inverse_transform takes in a binary representation of the classes and outputs its original form
# take way, we can view the results back
predicted_labels_inv = mlb.inverse_transform(predicted_labels_tfidf)

In [72]:
# run through a few examples
n = 5
# n_random_tests = [randint(0, len(X_test_view) - 1) for i in range(n)]
for i in range(10):
    print(f'Title: {X_test_view[i]}')
    print(f'True Tags: {y_test_view[i]}')
    print(f'Predicted Labels: {predicted_labels_inv[i]}')
    
print(randint(0, 9))

Title: caret rotating working
True Tags: ['asynchronous']
Predicted Labels: ()
Title: parallel job execution split-and-aggregate java
True Tags: ['java', 'android', 'database', 'button']
Predicted Labels: ()


KeyError: 2

In [ ]:
print(X_test[0])

In [ ]:
# TODO:
# - pipelining? https://stackoverflow.com/questions/42821315/userwarning-label-not-number-is-present-in-all-training-examples
